In [ ]:
DATASET_NAME = "ISBI2012_EM" # DRIVE, CVC_ClinicDB, LUNGS, ISIC2018
CFG_NAME = "Unet"
# Import libraries

import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline
import numpy as np
import time
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras import callbacks
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import models as M
import losses as l

In [ ]:
ROOT_DIR = os.path.abspath("./")
DATASET_FOLDER = "npys"
DATASET_PATH = os.path.join(ROOT_DIR, "datasets", DATASET_FOLDER)
EXPERIMENT_NAME = "{}_{}".format(DATASET_NAME, CFG_NAME)

if not os.path.exists(os.path.join(ROOT_DIR, "logs")):
    os.mkdir(os.path.join(ROOT_DIR, "logs"))

LOG_PATH = os.path.join(ROOT_DIR, "logs", EXPERIMENT_NAME)

if not os.path.exists(LOG_PATH):
    os.mkdir(LOG_PATH)
    
print(os.listdir(DATASET_PATH))

train_data = np.load("{}/{}_images.npy".format(DATASET_PATH, DATASET_NAME)) 
train_labels = np.load("{}/{}_mask.npy".format(DATASET_PATH, DATASET_NAME))
print(train_data.shape, train_labels.shape)

In [ ]:
# Build model with backbone
backbone_name = 'vgg16'
model = M.unet_backbone(backbone=backbone_name, input_size = (train_data.shape[1], 
            train_data.shape[2], train_data.shape[-1]))

model.summary()

In [ ]:
len(model.layers)

In [ ]:
# Build raw model
model = M.g_wide_unet(input_size = (train_data.shape[1], 
            train_data.shape[2], train_data.shape[-1]))

model.summary()

In [ ]:
# unet: 7.76M
# gaussunet skip nontrainable: 7.76M (no extra learable params)

# wide unet 9.13M
# wide gauss unet 9.13M (no new learnable weights) 

In [ ]:
import kernels as Kr

In [ ]:
kernel_size = 3
sigma = 0.5

In [ ]:
lpf = Kr.gauss_2D(shape=(kernel_size, kernel_size),sigma=sigma)
lpf

In [ ]:
plt.grid(False)
plt.imshow(lpf, cmap='plasma')
plt.colorbar()

In [ ]:
blur, outline, sharpen = Kr.get_kernel()

In [ ]:
plt.grid(False)
plt.imshow(blur, cmap='plasma')
plt.colorbar()

In [ ]:
LOG_PATH

In [ ]:
plt.grid(False)
plt.imshow(outline, cmap='plasma')
plt.colorbar()
#plt.axis('off')
plt.savefig("filter.pdf", bbox_inches='tight', dpi=100)
plt.show()

In [ ]:
plt.grid(False)
plt.imshow(sharpen, cmap='plasma')
plt.colorbar()